<a href="https://colab.research.google.com/github/aheiX/Teaching/blob/main/Layoutplanung_4x4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Layoutplanung 4x4

## Aufgabenstellung

Es soll für jede Stationen $T=[1,2,3,4]$ ein optimaler Standort $D=[A,B,C,D]$ gefunden werden. Die folgenden Informationen zur Distanz zwischen den Standorten und dem Materialfluss zwischen den Stationen ist gegeben:

<br>
$
\begin{array}{lrrrrrrr}
    \hline
    \text{Gramm pro Teilchen} & \text{Mehl} & \text{Zucker} & \text{Butter} & \text{Mandeln} & \text{Kakao} & \text{Eier} & \text{Verkaufspreis}  \\ \hline
     \text{Mandelschnitten} & 100 & 40 & 40 & 120 & - & 0,5 & 2,00 \\ 
     \text{Marmorschnecken} & 140 & 32 & 40 & - & 16 & 0,5 & 1,50 \\ 
     \text{Sandtörtchen} & 160 & 40 & 60 & - & - & 0,5 & 1,20 \\ 
     \text{Linzertaler} & 160 & 60 & - & 100 & 10 & 1,5 & 6,00 \\ \hline
     \text{Verfügbare Mengen} & 4.000 & 2.000 & 2.400 & 2.000 & 400 & 50 &  \\ 
     \hline
\end{array}
$
<br><br>

Die Kosten berechnen sich aus einem Einheitskostesatz $c=5$ pro Mengeneinheit und pro Distanzeinheit. Ziel ist es die Kosten zu minimieren.

Das Problem kann als klassisches Quadratisches Zuordnungproblem formuliert werden. Die Multiplikation der Entscheidungsvariablen wird durch das Einführung von Hilfsvariablen linearisiert (siehe Übungsaufgabe). 


## Mathematisches Modell

**Notation und Entscheidungsvariablen**<br>

\begin{array}{ll}
T & \text{Set an Stationen}\\	
D  & \text{Set an potentiellen Standorten}\\
m_{ij} & \text{Materialfluss zwischen zwei Stationen $i,j \in T$} \\
d_{ij}& \text{Distanz zwischen zwei Standorten $i,j \in D$}\\
c & \text{Einheitskostensatz}\\
x_{ij} & \text{=1, falls Station $i\in T$ am Standort $j \in D$ errichtet wird; =0, sonst}\\
z_{ijkl} & \text{Hilfsvariable für Linearisierung: =1, falls $x_{ik} \cdot x_{jl} = 1$, =0, sonst}\\
\end{array}
<br>

**Zielfunktion**<br><br>
\begin{array}{l}
\min c \cdot 
\sum\limits_{i \in T}
\sum\limits_{j \in T}
\sum\limits_{k \in D}
\sum\limits_{l \in D} m_{ij}
\cdot d_{kl} \cdot z_{ijkl}
\end{array}
<br><br>

**Nebenbedingungen**<br>

\begin{array}{rclll}
\sum\limits_{j \in D} x_{ij} & = & 1 &, \forall i \in T & \hspace{1.5cm}(1)\\
\sum\limits_{i \in D} x_{ij} & = & 1 &, \forall j \in T & \hspace{1.5cm}(2)\\
z_{ijkl} & \le & x_{ik} &, \forall i,j \in T \text{ und } k,l \in D & \hspace{1.5cm}(3) \text{ Linearisierungs NB 1}\\
z_{ijkl} & \ge & x_{ik} + x_{jl} - 1 &, \forall i,j \in T \text{ und } k,l \in D & \hspace{1.5cm}(4) \text{ Linearisierungs NB 2}\\
x_{ij}			   & \in & \mathbb{B} & , \forall i \in T, j \in D  & \\
z_{ijkl} & \in & \mathbb{B} &, \forall i,j \in T \text{ und } k,l \in D & \\
\end{array}


## Implementierung

In [4]:
# Daten als Dictionary
Standorte = ['A', 'B', 'C', 'D']
dist = dict()
dist['A'] = dict(A=0, B=60, C=80, D=20)
dist['B'] = dict(A=60, B=0, C=60, D=60)
dist['C'] = dict(A=80, B=60, C=0, D=100)
dist['D'] = dict(A=20, B=60, C=100, D=0)
# print(dist)

Stationen = ['S1', 'S2', 'S3', 'S4']
volume = dict()
volume['S1'] = dict(S1=0, S2=85, S3=20, S4=18)
volume['S2'] = dict(S1=30, S2=0, S3=3, S4=1)
volume['S3'] = dict(S1=32, S2=0, S3=0, S4=0)
volume['S4'] = dict(S1=27, S2=33, S3=8, S4=0)
# print(volume)

{'S1': {'S1': 0, 'S2': 85, 'S3': 20, 'S4': 18}, 'S2': {'S1': 30, 'S2': 0, 'S3': 3, 'S4': 1}, 'S3': {'S1': 32, 'S2': 0, 'S3': 0, 'S4': 0}, 'S4': {'S1': 27, 'S2': 33, 'S3': 8, 'S4': 0}}


In [15]:
# PuLP installieren
# !pip install pulp
# import pulp

# Modell erstellen
model = pulp.LpProblem(name='Layoutplanung_4x4', sense=pulp.constants.LpMinimize)

# Entscheidungsvariablen
x = pulp.LpVariable.dicts(name='x', indices=(Stationen, Standorte), cat='Binary')
z = pulp.LpVariable.dicts(name='z', 
                          indices=(Stationen, Stationen, Standorte, Standorte), 
                          cat='Binary')

# Zielfunktion
model += 5 * sum(volume[i][j]*dist[k][l]*z[i][j][k][l]
                 for i in Stationen 
                 for j in Stationen 
                 for k in Standorte 
                 for l in Standorte)

# Nebenbedingung 1
for i in Stationen:
  model += sum(x[i][j] for j in Standorte) == 1

# Nebenbedingung 2
for j in Standorte:
  model += sum(x[i][j] for i in Stationen) == 1

# Linearisierungs NB 1
for i in Stationen:
  for j in Stationen:
    for k in Standorte:
      for l in Standorte:
        model += z[i][j][k][l] <= x[i][k]

# Linearisierungs NB 2
for i in Stationen:
  for j in Stationen:
    for k in Standorte:
      for l in Standorte:
        model += z[i][j][k][l] >= x[i][k] + x[j][l] - 1


# Modell lösen
model.solve()

# Statistik
print('Status:', pulp.LpStatus[model.status])
print('Zielwert:', pulp.value(model.objective))
for v in model.variables():
  if v.varValue > 0:
    print(v.name, '=', v.varValue)

Status: Optimal
Zielwert: 59900.0
x_S1_A = 1.0
x_S2_D = 1.0
x_S3_C = 1.0
x_S4_B = 1.0
z_S1_S1_A_A = 1.0
z_S1_S2_A_D = 1.0
z_S1_S3_A_C = 1.0
z_S1_S4_A_B = 1.0
z_S2_S1_D_A = 1.0
z_S2_S2_D_D = 1.0
z_S2_S3_D_C = 1.0
z_S2_S4_D_B = 1.0
z_S3_S1_C_A = 1.0
z_S3_S2_C_D = 1.0
z_S3_S3_C_C = 1.0
z_S3_S4_C_B = 1.0
z_S3_S4_C_D = 1.0
z_S4_S1_B_A = 1.0
z_S4_S2_B_D = 1.0
z_S4_S3_B_C = 1.0
z_S4_S4_B_B = 1.0
